In [5]:
from pyspark import SparkConf, SparkContext

In [6]:
conf = SparkConf().setMaster('local').setAppName("Super hero graph")

In [9]:
# uncomment it when run as script
# sc = SparkContext(conf=conf)

sc = SparkContext.getOrCreate()
sc.appName="Super hero graph"
sc

<SparkContext master=local[*] appName=Super hero graph>

In [23]:
def parseNames(line):
    #1 "24-HOUR MAN/EMMANUEL"
    fields = line.split('\"')
    return (int(fields[0]) , fields[1].encode('utf8'))

def getCoOccurances(line):
    """
    return number of times a particular hero has appeared with other heroes
    """
    fields = line.split()
    # hero id and number of times he has appaered with various other heroes
    # -1 is to exclude the 1st field
    return (int(fields[0]) , len(fields) - 1) 

    

In [24]:
super_heroes = sc.textFile('./datasets/Marvel-Names.txt') 

In [25]:
super_hero_names = super_heroes.map(parseNames)

In [26]:
super_hero_graph = sc.textFile("./datasets/Marvel-Graph.txt")

In [27]:
super_hero_coOccurance = super_hero_graph.map(getCoOccurances)

In [28]:
super_hero_total_apperances = super_hero_coOccurance.reduceByKey(lambda x,y : x+y)

In [39]:
super_heroes_sorted = super_hero_total_apperances.sortBy(lambda x : x[1], ascending=False)
# max operations is performed on keys to flipping the rdd 
super_heroes_sorted_flipped  = super_heroes_sorted.map(lambda x :  x[::-1])
coApperance , popular_super_hero_id = super_heroes_sorted_flipped.max()

In [46]:
popular_super_hero_name = super_hero_names.lookup(popular_super_hero_id)[0].decode()
popular_super_hero_name

'CAPTAIN AMERICA'

In [48]:
print("%s is the most popular with %d apperances with co stars" % (popular_super_hero_name,coApperance))

CAPTAIN AMERICA is the most popular with 1933 apperances with co stars
